In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Manage_data.db_manager import get_data

In [2]:
df = get_data('META', granularity='daily')
df.head()

Adding Symbol: META
entry already exists or no title


,Title,Date,Open,High,Low,Close,Adj_close,Volume
0,META,2025-02-28,658.04,669.63,642.60,668.20,668.20,17517800
1,META,2025-02-27,682.45,688.65,657.57,658.24,658.24,12500000
2,META,2025-02-26,659.65,683.01,658.00,673.70,673.70,14488700
3,META,2025-02-25,665.97,668.00,641.86,657.50,657.50,20579700
4,META,2025-02-24,686.28,687.27,662.45,668.13,668.13,15677000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Title      3214 non-null   object        
 1   Date       3214 non-null   datetime64[ns]
 2   Open       3214 non-null   float64       
 3   High       3214 non-null   float64       
 4   Low        3214 non-null   float64       
 5   Close      3214 non-null   float64       
 6   Adj_close  3214 non-null   float64       
 7   Volume     3214 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 201.0+ KB


In [8]:
train_set = df.iloc[:, 1:2].values
print(train_set)
print(train_set.shape)

[['2025-02-28T00:00:00.000000000']
 ['2025-02-27T00:00:00.000000000']
 ['2025-02-26T00:00:00.000000000']
 ...
 ['2012-05-22T00:00:00.000000000']
 ['2012-05-21T00:00:00.000000000']
 ['2012-05-18T00:00:00.000000000']]
(3214, 1)


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range= (0,1))
scaled_train_set = scaler.fit_transform(train_set)
scaled_train_set

array([[1.00000000e+00],
       [9.99785821e-01],
       [9.99571643e-01],
       ...,
       [8.56714500e-04],
       [6.42535875e-04],
       [0.00000000e+00]], shape=(3214, 1))

In [10]:
X_train = []
y_train = []
for i in range(30, 1200):
    X_train.append(scaled_train_set[i-30:i, 0])
    y_train.append(scaled_train_set[i, 0])
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train)
print(X_train.shape)
print(y_train)
print(y_train.shape)

[[1.         0.99978582 0.99957164 ... 0.99186121 0.9910045  0.99079032]
 [0.99978582 0.99957164 0.99935746 ... 0.9910045  0.99079032 0.99057614]
 [0.99957164 0.99935746 0.99914329 ... 0.99079032 0.99057614 0.99036196]
 ...
 [0.63675305 0.63653887 0.63632469 ... 0.62818591 0.62797173 0.62775755]
 [0.63653887 0.63632469 0.63546798 ... 0.62797173 0.62775755 0.62754337]
 [0.63632469 0.63546798 0.6352538  ... 0.62775755 0.62754337 0.62668666]]
(1170, 30)
[0.99057614 0.99036196 0.99014778 ... 0.62754337 0.62668666 0.62647248]
(1170,)


In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_train.shape

(1170, 30, 1)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=4, dropout=0.2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out[:, -1, :])  # Prendre la dernière sortie temporelle
        output = self.fc(lstm_out)
        return output

input_size = 1
model = LSTMModel(input_size=input_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


AttributeError: 'numpy.ndarray' object has no attribute 'dim'